<a href="https://colab.research.google.com/github/ganesh-207/Machine_learning/blob/main/Machine_Failure_Predition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Failure Project

## Importing all the necessary libraries

In [82]:
# ignore non-harmful warnings
from warnings import filterwarnings
filterwarnings('ignore')

# To read the dataset
import pandas as pd

# To clean the data preprocess it
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder

# To Visualize and ploting
import seaborn as sns
import matplotlib.pyplot as plt

# To ploting
import matplotlib.pyplot as plt
import seaborn as sns

# Divide the data into training and testing
from sklearn.model_selection import train_test_split,cross_val_score

# Model building=> Decision Tree, Logistic Regression, Random Forest, Gradient Boost,XGBoost
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve,auc

# Evaluation metrics
from sklearn.metrics import f1_score,ConfusionMatrixDisplay,classification_report,accuracy_score

## Read the dataset

In [14]:
path = r'https://raw.githubusercontent.com/ganesh-207/Datasets/refs/heads/main/Machine_train.csv'
df = pd.read_csv(path)
df.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [15]:
print(f'Shape of Data {df.shape}')
print(f'Columns Present In Dataset: {df.columns}')
print(f'Shape of Data: {df.info()}')

Shape of Data (136429, 14)
Columns Present In Dataset: Index(['id', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       136429 non-null  int64  
 1   Product ID               136429 non-null  object 
 2   Type                     136429 non-null  object 
 3   Air temperature [K]      136429 non-null  float64
 4   Process temperature [K]  136429 non-null  float64
 5   Rotational speed [rpm]   136429 non-null  int64  
 6   Torque [Nm]              136429 non-null  float64
 7   Tool wear [min]          136429 non-null  int64  
 8   Machine failure          136429 non-

In [16]:
df.columns=['id', 'Product ID', 'Type', 'Air temperature (K)',
       'Process temperature (K)', 'Rotational speed (rpm)', 'Torque (Nm)',
       'Tool wear (min)', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF']

In [17]:

df['Machine failure'].value_counts()

,count
Machine failure,
0,134281
1,2148


## Perform basic data quality checks

In [18]:
# Missing Values
print(f'Missing Values ')
df.isna().sum()

Missing Values 


,0
id,0
Product ID,0
Type,0
Air temperature (K),0
Process temperature (K),0
Rotational speed (rpm),0
Torque (Nm),0
Tool wear (min),0
Machine failure,0
TWF,0


In [19]:
## Duplicated rows
print(f'Duplicated rows')
df.duplicated().sum()

Duplicated rows


np.int64(0)

## There are no missing values nor any duplicates in this dataset

In [20]:
df = df.drop_duplicates()
print(f'Duplicated rows are dropped')

Duplicated rows are dropped


## Separate X and Y features
    Y: Machine Failure
    X: Remaining all the features

In [21]:
X = df.drop(columns=['id','Machine failure'])
Y = df[['Machine failure']]
print(X.head())
print('-----------')
print(Y.head())

  Product ID Type  Air temperature (K)  Process temperature (K)  \
0     L50096    L                300.6                    309.6   
1     M20343    M                302.6                    312.1   
2     L49454    L                299.3                    308.5   
3     L53355    L                301.0                    310.9   
4     M24050    M                298.0                    309.0   

   Rotational speed (rpm)  Torque (Nm)  Tool wear (min)  TWF  HDF  PWF  OSF  \
0                    1596         36.1              140    0    0    0    0   
1                    1759         29.1              200    0    0    0    0   
2                    1805         26.5               25    0    0    0    0   
3                    1524         44.3              197    0    0    0    0   
4                    1641         35.4               34    0    0    0    0   

   RNF  
0    0  
1    0  
2    0  
3    0  
4    0  
-----------
   Machine failure
0                0
1                0

## Data Preprocessing and Data Cleaning

In [22]:
cat = list(X.columns[X.dtypes == 'object'])
con = list(X.columns[X.dtypes != 'object'])

In [23]:
# Create a categorical pipeline that handles missing data and also scales text data
cat_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'),OrdinalEncoder())
# Create a numerical pipeline that handles missing data and also scales numerical data
con_pipe = make_pipeline(SimpleImputer(strategy='mean'),StandardScaler())
#Combine both the piplines
pre = ColumnTransformer([
    ('cat',cat_pipe,cat),
    ('con',con_pipe,con)
]).set_output(transform='pandas')

In [24]:
pre

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder())]),
                                 ['Product ID', 'Type']),
                                ('con',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Air temperature (K)',
                                  'Process temperature (K)',
                                  'Rotational speed (rpm)', 'Torque (Nm)',
                                  'Tool wear (min)', 'TWF', 'HDF', 'PWF', 'OSF',
                                  'RNF'])])

In [25]:
X_pre = pre.fit_transform(X)

In [26]:
X_pre.head()

,cat__Product ID,cat__Type,con__Air temperature (K),con__Process temperature (K),con__Rotational speed (rpm),con__Torque (Nm),con__Tool wear (min),con__TWF,con__HDF,con__PWF,con__OSF,con__RNF
0,2752.0,1.0,0.395880,-0.246230,0.545416,-0.499711,0.556417,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
1,8620.0,2.0,1.469856,1.558605,1.720308,-1.323028,1.494433,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
2,2344.0,1.0,-0.302204,-1.040358,2.051873,-1.628831,-1.241447,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
3,4704.0,1.0,0.610675,0.692284,0.026445,0.464745,1.447532,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
4,9727.0,2.0,-1.000288,-0.679391,0.869773,-0.582043,-1.100744,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568


## Splitting of data into training and testing

In [27]:
xtrain,xtest,ytrain,ytest = train_test_split(X_pre,Y,train_size=0.75,random_state=21)
print(f'xtrain shape: {xtrain.shape}')
print(f'xtest shape: {xtest.shape}')
print(f'ytrain shape: {ytrain.shape}')
print(f'ytest shape: {ytest.shape}')

xtrain shape: (102321, 12)
xtest shape: (34108, 12)
ytrain shape: (102321, 1)
ytest shape: (34108, 1)


In [28]:
from imblearn.under_sampling import RandomUnderSampler

In [29]:
rus = RandomUnderSampler()
xtrain_res,ytrain_res = rus.fit_resample(xtrain,ytrain)

In [40]:
print("Before:",xtrain.value_counts())
print("After:", xtrain_res.value_counts())

Before: cat__Product ID  cat__Type  con__Air temperature (K)  con__Process temperature (K)  con__Rotational speed (rpm)  con__Torque (Nm)  con__Tool wear (min)  con__TWF  con__HDF   con__PWF   con__OSF   con__RNF 
4655.0           1.0         0.556977                  0.547897                     -0.780843                     2.017285         -1.382149             -0.03945  -0.072021  -0.049016  -0.063038  -0.047568    30
5174.0           1.0         0.449579                  0.764477                     -0.773635                     1.370394         -0.209629             -0.03945  -0.072021  -0.049016  -0.063038  -0.047568    12
1761.0           1.0        -0.678095                 -0.101844                     -0.110506                     0.629409          0.618951             -0.03945  -0.072021  -0.049016  -0.063038  -0.047568     9
8715.0           2.0         0.825470                  0.764477                     -1.343061                     1.523295          0.822188          

## Algorithm Evaluation

In [94]:
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    XGBClassifier()
]

In [31]:
models[0]

LogisticRegression()

In [32]:
type(models[0])

sklearn.linear_model._logistic.LogisticRegression

In [33]:
type(models[0]).__name__

'LogisticRegression'

In [75]:
def evaluate_model_performance(model,xtrain,ytrain,xtest,ytest):
    # Fit the model
    model.fit(xtrain_res,ytrain_res)

    # Predict the outputs-Machine_failure
    ypred_train = model.predict(xtrain_res)
    ypred_test = model.predict(xtest)

    # Evaluation on basis of metrics: f1-score
    f1_train = f1_score(ytrain,ypred_train)
    f1_test = f1_score(ytest,ypred_test)

    # Evaluate through cross validation also
    scores = cross_val_score(model,xtrain_res,ytrain_res,cv=4,scoring='f1_macro')
    f1_cv = scores.mean()

     # Accuracy Score on train dataset
    accuracy_train = accuracy_score(ytrain_res,ypred_train)

    # Accuracy Score on test dataset
    accuracy_test = accuracy_score(ytest,ypred_test)

    # Returns all the above results for the model
    results = {
        'Name':type(model).__name__,
        'f1_score:training':f1_train,
        'f1_score:testing':f1_test,
        'f1_cv':f1_cv,
        'Accuracy_train_score':accuracy_train,
        'Accuracy_test_score':accuracy_test
    }
    return results

In [76]:
models

[LogisticRegression(),
 DecisionTreeClassifier(),
 RandomForestClassifier(),
 GradientBoostingClassifier(),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               feature_weights=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=None,
               n_jobs=None, num_parallel_tree=None, ...)]

In [77]:
def evaluate_model(models,xtrain_res,ytrain_res,xtest,ytest):
    res = []
    for i in models:
        r = evaluate_model_performance(i,xtrain_res,ytrain_res,xtest,ytest)

        # combine all the results of each model provided as input to above method
        res.append(r)
    # Convert these results into a dataframe and display
    results_df = pd.DataFrame(res)
    return results_df

In [78]:
score_results_df = evaluate_model(models,xtrain_res,ytrain_res,xtest,ytest)

In [79]:
score_results_df

,Name,f1_score:training,f1_score:testing,f1_cv,Accuracy_train_score,Accuracy_test_score
0,LogisticRegression,0.881344,0.621701,0.890439,0.892413,0.984872
1,DecisionTreeClassifier,0.999689,0.182311,0.855977,0.999689,0.876129
2,RandomForestClassifier,0.999689,0.320539,0.902276,0.999689,0.940835
3,GradientBoostingClassifier,0.922977,0.385260,0.904368,0.926306,0.956960
4,XGBClassifier,0.997197,0.312173,0.899790,0.997201,0.938372


In [80]:
score_results_df.sort_values(by='f1_cv',ascending=False)

,Name,f1_score:training,f1_score:testing,f1_cv,Accuracy_train_score,Accuracy_test_score
3,GradientBoostingClassifier,0.922977,0.385260,0.904368,0.926306,0.956960
2,RandomForestClassifier,0.999689,0.320539,0.902276,0.999689,0.940835
4,XGBClassifier,0.997197,0.312173,0.899790,0.997201,0.938372
0,LogisticRegression,0.881344,0.621701,0.890439,0.892413,0.984872
1,DecisionTreeClassifier,0.999689,0.182311,0.855977,0.999689,0.876129
